## Visulization 

In [ ]:
# Import necessary library
import pandas as pd
import folium


# Load and Clean Data

file_path = "path_to_file/earthquake_data.csv"  # Adjust file path as needed
data = pd.read_csv(file_path)

data.dropna(subset=['longitude', 'latitude', 'magnitude', 'depth'], inplace=True)



# Filter rows where 'eventtype' is equal to 'earthquake'
data = data[data['eventtype'] == 'earthquake']



# Ensure the 'origintime' column is in datetime format for temporal analysis
data['origintime'] = pd.to_datetime(data['origintime'], errors='coerce')

### 1- Earthquake Scatter Map

In [ ]:
# Define a function to create a clean, minimalistic earthquake scatter map
def create_clean_scatter_map(data, save_path="earthquake_map.html"):
    """
    Create an interactive scatter map to visualize earthquake data.

    Parameters:
    - data (DataFrame): Data containing 'latitude', 'longitude', 'magnitude', and 'depth'.
    - save_path (str): Path to save the generated HTML file (default: 'earthquake_map.html').

    Returns:
    - None: Saves the map as an HTML file.
    """
    # Step 1: Initialize the map
    # Center the map on New Zealand with a starting zoom level of 6.
    m = folium.Map(
        location=[-40.9006, 174.886],  # Coordinates of New Zealand
        zoom_start=6  # Initial zoom level
    )

    # Step 2: Add a clean and minimalistic tile layer
    # Use CartoDB's "positron" tile layer for a visually appealing map background.
    folium.TileLayer('CartoDB positron').add_to(m)

    # Step 3: Add earthquake data as circle markers on the map
    # Loop through each row in the dataset and plot earthquake locations.
    for _, row in data.iterrows():
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],  # Earthquake coordinates
            radius=row['magnitude'] * 2,  # Circle size based on magnitude
            color="red",  # Circle border color
            fill=True,  # Enable fill for the circle
            fill_opacity=0.7,  # Set fill opacity for visibility
            popup=f"Magnitude: {row['magnitude']}, Depth: {row['depth']} km"  # Information popup
        ).add_to(m)

    # Step 4: Apply custom CSS for aesthetics
    # Add a custom style to remove additional grey space and match the ocean color.
    m.get_root().html.add_child(folium.Element("""
    <style>
        .leaflet-container {
            background: #e5f3ff;  /* Ocean-like background color */
            border: none !important;  /* Remove border for a clean look */
        }
    </style>
    """))

    # Step 5: Save the map as an HTML file
    # Save the generated map to the specified file path.
    m.save(save_path)
    print(f"Clean map saved as {save_path}")

# Example usage:
# Call the function to generate the map and save it to an HTML file.
create_clean_scatter_map(data)


## 2- Earthquake heat map

In [ ]:
from folium.plugins import HeatMap

# Create heatmap for earthquake density
def create_earthquake_heatmap(data, save_path="heatmap.html"):
    """
    Generate a heatmap to visualize earthquake density.

    Parameters:
    - data (DataFrame): Data containing 'latitude' and 'longitude' columns.
    - save_path (str): Path to save the generated HTML file (default: 'heatmap.html').
    """
    # Initialize the map centered on New Zealand with an appropriate zoom level
    m = folium.Map(location=[-40.9006, 174.886], zoom_start=8)

    # Prepare the data for the heatmap
    # Extract latitude and longitude for each earthquake in the dataset
    heat_data = [[row['latitude'], row['longitude']] for _, row in data.iterrows()]

    # Add the heatmap layer to the map
    # The heatmap will highlight regions with higher earthquake densities
    HeatMap(heat_data).add_to(m)

    # Add a clean, minimalistic tile layer for better visualization
    folium.TileLayer('CartoDB positron').add_to(m)

    # Save the generated heatmap as an interactive HTML file
    m.save(save_path)
    print(f"Heatmap saved as {save_path}")  # Confirmation message

# Call the function with the earthquake data
create_earthquake_heatmap(data)


## 3- Spatial Distribution: Show clusters of earthquakes

In [ ]:
# Spatial Distribution: Show clusters of earthquakes
def plot_spatial_distribution(data):
    plt.figure(figsize=(10, 6))
    plt.scatter(data['longitude'], data['latitude'], c=data['magnitude'], cmap='Reds', s=10)
    plt.colorbar(label='Magnitude')
    plt.title('Spatial Distribution of Earthquakes')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.show()

plot_spatial_distribution(data)

## 4-  Magnitude Patterns: Magnitude vs Location

In [ ]:
# Magnitude Patterns: Magnitude vs Location
def plot_magnitude_vs_location(data):
    plt.figure(figsize=(10, 6))
    sns.scatterplot(data=data, x='longitude', y='latitude', hue='magnitude', palette='coolwarm', size='magnitude', sizes=(20, 200))
    plt.title('Magnitude vs Location of Earthquakes')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.legend(title='Magnitude', loc='upper right')
    plt.show()

plot_magnitude_vs_location(data)

## 5- Depth Analysis: Depth vs Magnitude (Scatterplot)

In [ ]:
# 3. Depth Analysis: Depth vs Magnitude (Scatterplot)
def plot_depth_vs_magnitude(data):
    plt.figure(figsize=(10, 6))
    sns.scatterplot(data=data, x='depth', y='magnitude', hue='depth', palette='viridis')
    plt.title('Depth vs Magnitude of Earthquakes')
    plt.xlabel('Depth (km)')
    plt.ylabel('Magnitude')
    plt.show()


plot_depth_vs_magnitude(data)


## 6- Temporal Patterns: Earthquake frequency by month

In [ ]:
# Temporal Patterns: Earthquake frequency by month
def plot_temporal_patterns(data):
    data['month'] = data['origintime'].dt.month
    monthly_counts = data['month'].value_counts().sort_index()
    
    plt.figure(figsize=(10, 6))
    sns.lineplot(x=monthly_counts.index, y=monthly_counts.values, marker='o')
    plt.title('Earthquake Frequency by Month')
    plt.xlabel('Month')
    plt.ylabel('Number of Earthquakes')
    plt.xticks(range(1, 13))
    plt.show()
plot_temporal_patterns(data)

## 7- Intensity Visualization: Magnitude Radius

In [ ]:

#Intensity Visualization: Magnitude Radius
def plot_magnitude_radius(data):
    plt.figure(figsize=(10, 6))
    sns.histplot(data['magnitude'], kde=True, color='red')
    plt.title('Distribution of Earthquake Magnitudes')
    plt.xlabel('Magnitude')
    plt.ylabel('Frequency')
    plt.show()

plot_magnitude_radius(data)

## 8- Comparison by Location: Earthquakes in North vs South Island

In [ ]:

#Comparison by Location: Earthquakes in North vs South Island
def plot_north_vs_south(data):
    data['region'] = data['latitude'].apply(lambda x: 'North Island' if x > -41 else 'South Island')
    region_counts = data['region'].value_counts()
    
    plt.figure(figsize=(10, 6))
    sns.barplot(x=region_counts.index, y=region_counts.values, palette='Set2')
    plt.title('Earthquakes in North vs South Island')
    plt.xlabel('Region')
    plt.ylabel('Number of Earthquakes')
    plt.show()

plot_north_vs_south(data)

## 9- Depth vs Magnitude Correlation: Plot depth against magnitude

In [ ]:

# Depth vs Magnitude Correlation: Plot depth against magnitude
def plot_depth_vs_magnitude_corr(data):
    plt.figure(figsize=(10, 6))
    sns.scatterplot(data=data, x='depth', y='magnitude', hue='magnitude', palette='coolwarm')
    plt.title('Depth vs Magnitude Correlation')
    plt.xlabel('Depth (km)')
    plt.ylabel('Magnitude')
    plt.show()

plot_depth_vs_magnitude_corr(data)